ESQUELETO:

Este es el archivo para subir el modelo a la nube.


CÓMO FUNCIONA:

1. Tomar un modelo ya realizado.

2. Lo sube a la nube.

ASEGÚRENSE DE QUE SE COORDINAN LO NECESARIO PARA QUE LOS NOMBRES COINCIDAN (no es de chill cambiar el estándar xd).

In [4]:
import sys 
!{sys.executable} -m pip install azureml-core

  Using cached azureml_core-1.59.0.post2-py3-none-any.whl.metadata (3.4 kB)
  Using cached backports.tempfile-1.0-py2.py3-none-any.whl.metadata (2.3 kB)
  Using cached pathspec-0.12.1-py3-none-any.whl.metadata (21 kB)
  Using cached azure_core-1.32.0-py3-none-any.whl.metadata (39 kB)
  Using cached paramiko-3.5.1-py3-none-any.whl.metadata (4.6 kB)
  Using cached azure_mgmt_resource-23.3.0-py3-none-any.whl.metadata (41 kB)
  Using cached azure_mgmt_authorization-4.0.0-py3-none-any.whl.metadata (18 kB)
  Using cached azure_graphrbac-0.61.2-py2.py3-none-any.whl.metadata (11 kB)
  Using cached azure_common-1.1.28-py2.py3-none-any.whl.metadata (5.0 kB)
  Using cached msrest-0.7.1-py3-none-any.whl.metadata (21 kB)
  Using cached msrestazure-0.6.4.post1-py2.py3-none-any.whl.metadata (15 kB)
  Using cached ndg_httpsclient-0.5.1-py3-none-any.whl.metadata (6.2 kB)
  Using cached SecretStorage-3.3.3-py3-none-any.whl.metadata (4.0 kB)
  Using cached jsonpickle-4.0.5-py3-none-any.whl.metadata (8.2 

In [5]:
## STEP 1: Set up the workspace.

import json

# #GETTING MY ID:
# id = open('my_id.json', 'r')
# mi = json.load(id)
# my_id = mi["my_id"],

from azureml.core import Workspace

ws = Workspace.create(name="workspace",
                      subscription_id = "b762ea09-6d96-40bc-a428-cdd2e87aa18c",
                      resource_group = "class2",
                      location = "centralindia")

from azureml.core.model import Model

mname = "model"
registered_model = Model.register(model_path="model_logreg.pkl",
                                  model_name=mname,
                                  workspace=ws)

Deploying AppInsights with name workspacinsightsfdb302d1.
Deployed AppInsights with name workspacinsightsfdb302d1. Took 7.66 seconds.
Deploying KeyVault with name workspackeyvaultb67ff004.
Deploying StorageAccount with name workspacstoragefdd925f67.
Deployed KeyVault with name workspackeyvaultb67ff004. Took 26.81 seconds.
Deploying Workspace with name workspace.
Deployed StorageAccount with name workspacstoragefdd925f67. Took 40.24 seconds.
Deployed Workspace with name workspace. Took 42.42 seconds.
Registering model model


In [10]:
################ score.py ###################

scorepy = f"""
import json
import joblib
import numpy as np
import pandas as pd
from azureml.core.model import Model

def init():
  global model
  model_path = Model.get_model_path('{mname}')
  model = joblib.load(model_path)

def run(raw_data):
  try: ## Try la predicción.
    data = json.loads(raw_data)['data'][0]
    data = pd.DataFrame(data)

    data_dummies = data[['NetIncometoTotalAssets', 'ROA(A)beforeinterestand%aftertax', 'ROA(B)beforeinterestanddepreciationaftertax',
 'ROA(C)beforeinterestanddepreciationbeforeinterest', 'Networth/Assets',  'Debtratio%', 'PersistentEPSintheLastFourSeasons',
 'RetainedEarningstoTotalAssets', 'Netprofitbeforetax/Paid-incapital',
 'PerShareNetprofitbeforetax(Yuan¥)']] ## INCLUIR AQUÍ LO DE ZORROUNO

    result = model.predict(data_dummies).tolist()

    return json.dumps(result)
  except Exception as e:
    return json.dumps(str(e))
"""

file_score = open("score.py", "w")
file_score.write(scorepy)
file_score.close()

In [11]:
## STEP 2: Inference configuration: the blueprints for the German carpenter about how to build your furniture.

from azureml.core.environment import Environment
virtual_env = Environment("env-4-housing")

from azureml.core.conda_dependencies import CondaDependencies
virtual_env.python.conda_dependencies = CondaDependencies.create(conda_packages=['pandas','scikit-learn']) ## Asegúrense de coordinarse con el departamento de modelos para incluir los paquetes correctos. Si no, BOOM!


from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice
inference_config = InferenceConfig(
                                environment=virtual_env,
                                entry_script="score.py",
                                )
aci_config = AciWebservice.deploy_configuration(cpu_cores=0.5, memory_gb=1) ## ASEGÚRENSE DE ASIGNAR SUFICIENTE MADERA PARA SUS MUEBLES.

service = Model.deploy(workspace=ws,
                       name='service',
                       models=[registered_model],
                       inference_config=inference_config,
                       deployment_config=aci_config,
                       overwrite=True,
                       )

C:\Users\crist\AppData\Local\Temp\ipykernel_8776\944137596.py:18: FutureWarning: azureml.core.model:
To leverage new model deployment capabilities, AzureML recommends using CLI/SDK v2 to deploy models as online endpoint, 
please refer to respective documentations 
https://docs.microsoft.com/azure/machine-learning/how-to-deploy-managed-online-endpoints /
https://docs.microsoft.com/azure/machine-learning/how-to-attach-kubernetes-anywhere 
For more information on migration, see https://aka.ms/acimoemigration 
To disable CLI/SDK v1 deprecation warning set AZUREML_LOG_DEPRECATION_WARNING_ENABLED to 'False'
  service = Model.deploy(workspace=ws,


In [12]:
service.wait_for_deployment()

scoring_uri = service.scoring_uri

scoreuri = json.dumps({"URI": [scoring_uri]})
file = open("uri.json", "w")
file.write(scoreuri)
file.close()

ACI service creation operation finished, operation "Succeeded"
